# 第 6 章: Trino

## Trinoでデータ処理を実行する

Trino は複数のデータソースに対する統一的な SQL インターフェースを提供する分散 SQL クエリエンジンです。大規模なデータウェアハウスからデータレイク、さらには複数のリレーショナルデータベースまで、異なるデータソースを単一のクエリで結合できます。

### 1. 初期設定

In [ ]:
# 必要なライブラリのインストール
!pip install -q trino pandas

In [ ]:
# 環境変数の設定
TRINO_URI = "http://trino:8085"
WAREHOUSE = "s3://amzn-s3-demo-bucket"

### 2. Trinoへの接続

Trino Python クライアントを使用して、Trino サーバーに接続します。

In [ ]:
from trino.dbapi import connect
import pandas as pd

# Icebergカタログを使用してTrinoに接続
conn = connect(host=TRINO_URI, user="trino", catalog="iceberg")
cur = conn.cursor()

# カタログの確認
cur.execute("SHOW CATALOGS")
catalogs = cur.fetchall()
print("利用可能なカタログ:")
for catalog in catalogs:
    print(catalog[0])

## Iceberg の利用を開始する

### 1. テーブルの作成と変更

まずは基本的なテーブル作成と変更のオペレーションを見ていきましょう。

In [ ]:
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.customers (
    customer_id BIGINT,
    name VARCHAR,
    email VARCHAR,
    registration_date DATE,
    segment VARCHAR,
    active BOOLEAN
)
""")

# テーブル情報の確認
cur.execute("DESCRIBE iceberg.retail.customers")
table_schema = cur.fetchall()
pd.DataFrame(table_schema, columns=['Column', 'Type', 'Extra', 'Comment'])

In [ ]:
# テーブル変更：新しい列の追加
cur.execute("ALTER TABLE iceberg.retail.customers ADD COLUMN birth_date DATE")
cur.execute("DESCRIBE iceberg.retail.customers")
updated_schema = cur.fetchall()
pd.DataFrame(updated_schema, columns=['Column', 'Type', 'Extra', 'Comment'])

In [ ]:
# テーブル変更：列の型変更
cur.execute("ALTER TABLE iceberg.retail.customers ALTER COLUMN name SET DATA TYPE VARCHAR(100)")

# テーブル変更：列名の変更
cur.execute("ALTER TABLE iceberg.retail.customers RENAME COLUMN email TO email_address")

# 最終的なテーブルスキーマの確認
cur.execute("DESCRIBE iceberg.retail.customers")
final_schema = cur.fetchall()
pd.DataFrame(final_schema, columns=['Column', 'Type', 'Extra', 'Comment'])

### 2. CREATE TABLE AS SELECTを使用したテーブル作成

Trino では、既存のクエリ結果から直接新しい Iceberg テーブルを作成できる「CREATE TABLE AS SELECT」（CTAS）構文をサポートしています。

In [ ]:
# CTAS　の元になるテーブルを作成
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.sales_by_date (
    sale_id VARCHAR,
    product_id VARCHAR,
    customer_id BIGINT,
    sale_date DATE,
    quantity INTEGER,
    amount DECIMAL(10, 2),
    region VARCHAR
)
""")

# データの挿入
cur.execute("""
INSERT INTO iceberg.retail.sales_by_date VALUES
    ('S-1001', 'P-101', 1001, DATE '2023-01-15', 2, 129.99, 'North'),
    ('S-1002', 'P-102', 1002, DATE '2023-01-15', 1, 89.95, 'North'),
    ('S-1003', 'P-101', 1003, DATE '2023-01-16', 3, 194.98, 'South'),
    ('S-1004', 'P-103', 1001, DATE '2023-02-01', 1, 49.99, 'North'),
    ('S-1005', 'P-102', 1004, DATE '2023-02-01', 2, 179.90, 'East'),
    ('S-1006', 'P-101', 1002, DATE '2023-02-15', 1, 64.99, 'North'),
    ('S-1007', 'P-103', 1003, DATE '2023-02-20', 4, 199.96, 'South'),
    ('S-1008', 'P-104', 1005, DATE '2023-03-01', 2, 149.98, 'West'),
    ('S-1009', 'P-102', 1002, DATE '2023-03-05', 3, 269.85, 'North'),
    ('S-1010', 'P-101', 1001, DATE '2023-03-15', 1, 64.99, 'North')
""")

# データの確認
cur.execute("SELECT * FROM iceberg.retail.sales_by_date")
products = cur.fetchall()
columns = [desc[0] for desc in cur.description]
pd.DataFrame(products, columns=columns)

In [ ]:
# sales_by_dateテーブルから月次売上サマリーを作成
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.monthly_sales_summary
WITH (
    format = 'PARQUET',
    partitioning = ARRAY['region']
)
AS
SELECT 
    date_trunc('month', sale_date) AS month,
    region,
    COUNT(*) AS transaction_count,
    SUM(amount) AS total_sales
FROM iceberg.retail.sales_by_date
GROUP BY date_trunc('month', sale_date), region
""")

# 作成されたテーブルのデータを確認
cur.execute("SELECT * FROM iceberg.retail.monthly_sales_summary ORDER BY month, region")
summary_data = cur.fetchall()
columns = [desc[0] for desc in cur.description]
pd.DataFrame(summary_data, columns=columns)

### 3. パーティションを利用したテーブルの作成

Icebergはパーティショニングをサポートしています。ここでは、注文データをステータスでパーティショニングしたテーブルを作成します。

In [ ]:
# 注文テーブルの作成（ステータスでパーティショニング）
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.orders (
    order_id VARCHAR,
    customer_id BIGINT,
    order_date DATE,
    total_amount DECIMAL(10, 2),
    status VARCHAR
)
WITH (
    partitioning = ARRAY['status']
)
""")

# サンプルデータの挿入
cur.execute("""
INSERT INTO iceberg.retail.orders VALUES
    ('O-1001', 101, DATE '2023-02-01', 149.99, 'COMPLETED'),
    ('O-1002', 102, DATE '2023-02-01', 89.95, 'PROCESSING'),
    ('O-1003', 101, DATE '2023-02-02', 32.50, 'SHIPPED'),
    ('O-1004', 103, DATE '2023-02-03', 199.99, 'PROCESSING'),
    ('O-1005', 101, DATE '2023-02-05', 75.00, 'COMPLETED')
""")

# データの確認
cur.execute("SELECT * FROM iceberg.retail.orders")
orders = cur.fetchall()
order_columns = [desc[0] for desc in cur.description]
pd.DataFrame(orders, columns=order_columns)

Icebergのメタデータテーブルを使って、パーティション情報を確認できます。

In [ ]:
# パーティション情報の確認
cur.execute("SELECT * FROM iceberg.retail.\"orders$partitions\"")
partitions = cur.fetchall()
partition_columns = [desc[0] for desc in cur.description]
pd.DataFrame(partitions, columns=partition_columns)

パーティション列を使った集計クエリを実行します。パーティショニングにより、特定のステータスに関するクエリが効率的に実行されます。

In [ ]:
# 注文ステータス別の集計
cur.execute("""
SELECT 
    status,
    COUNT(*) AS order_count,
    SUM(total_amount) AS total_sales
FROM iceberg.retail.orders
GROUP BY status
""")
status_summary = cur.fetchall()
status_columns = [desc[0] for desc in cur.description]
pd.DataFrame(status_summary, columns=status_columns)

### 4. パーティション変換(Transform) を使用したパーティショニング
Iceberg はシンプルな列ベースのパーティショニングだけでなく、Transform を用いた高度なパーティショニング戦略もサポートしています。これらの関数を使うことで、データ分布に最適化されたパーティショニングスキーマを構築できます。

- 時間関連の変換関数: year(), month(), day(), hour()
- バケッティング関数: bucket(バケット数, 列名)
- 値の切り捨て関数: truncate(長さ, 列名)


In [ ]:
# 1. 時間階層パーティショニング - 年と月でパーティション分割
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.time_hierarchical_sales (
    sale_id VARCHAR,
    product_id VARCHAR,
    customer_id BIGINT,
    sale_timestamp TIMESTAMP,
    amount DECIMAL(10, 2)
)
WITH (
    format = 'PARQUET',
    partitioning = ARRAY['month(sale_timestamp)']
)
""")

# 2. 数値範囲のバケッティング - 金額を範囲でバケット化
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.amount_bucketed_sales (
    sale_id VARCHAR,
    product_id VARCHAR,
    customer_id BIGINT,
    sale_date DATE,
    amount DECIMAL(10, 2)
)
WITH (
    partitioning = ARRAY['truncate(amount,1000)']  -- 1000単位で金額をグループ化
)
""")

# 3. 複合パーティショニング - 地域によるパーティショニングと顧客IDのハッシュバケッティング
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.region_customer_sales (
    sale_id VARCHAR,
    product_id VARCHAR,
    customer_id BIGINT,
    sale_date DATE,
    region VARCHAR,
    amount DECIMAL(10, 2)
)
WITH (
    -- 地域によるパーティショニングと顧客 ID のバケット分割を設定
    partitioning = ARRAY['region', 'bucket(customer_id, 10)']
)
""")

## 読み込みオペレーション

### 1. 基本的なクエリと条件付きフィルタリング

テーブルデータの基本的な読み取りとフィルタリングの例を見てみましょう：

In [ ]:
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.sales_by_date (
    sale_id VARCHAR,
    product_id VARCHAR,
    customer_id BIGINT,
    sale_date DATE,
    quantity INTEGER,
    amount DECIMAL(10, 2),
    region VARCHAR
)
WITH (
    format = 'PARQUET',
    partitioning = ARRAY['sale_date']
)
""")

# データの挿入
cur.execute("""
INSERT INTO iceberg.retail.sales_by_date VALUES
    ('S-1001', 'P-101', 1001, DATE '2023-01-15', 2, 129.99, 'North'),
    ('S-1002', 'P-102', 1002, DATE '2023-01-15', 1, 89.95, 'North'),
    ('S-1003', 'P-101', 1003, DATE '2023-01-16', 3, 194.98, 'South'),
    ('S-1004', 'P-103', 1001, DATE '2023-02-01', 1, 49.99, 'North'),
    ('S-1005', 'P-102', 1004, DATE '2023-02-01', 2, 179.90, 'East'),
    ('S-1006', 'P-101', 1002, DATE '2023-02-15', 1, 64.99, 'North'),
    ('S-1007', 'P-103', 1003, DATE '2023-02-20', 4, 199.96, 'South'),
    ('S-1008', 'P-104', 1005, DATE '2023-03-01', 2, 149.98, 'West'),
    ('S-1009', 'P-102', 1002, DATE '2023-03-05', 3, 269.85, 'North'),
    ('S-1010', 'P-101', 1001, DATE '2023-03-15', 1, 64.99, 'North')
""")

# データの確認
cur.execute("SELECT * FROM iceberg.retail.sales_by_date")
products = cur.fetchall()
columns = [desc[0] for desc in cur.description]
pd.DataFrame(products, columns=columns)

In [ ]:
# 日付によるフィルタリング
# 2023年2月以降の販売データを取得
cur.execute("""
-- 日付によるフィルタリング
-- 2023年2月以降の販売データを取得
SELECT
    sale_date,
    COUNT(*) AS sale_count,
    SUM(quantity) AS total_quantity,
    SUM(amount) AS total_amount
FROM iceberg.retail.sales_by_date
WHERE sale_date >= DATE '2023-02-01'
GROUP BY sale_date
ORDER BY sale_date
""")
filtered_data = cur.fetchall()
filtered_columns = [desc[0] for desc in cur.description]
pd.DataFrame(filtered_data, columns=filtered_columns)

In [ ]:
# 複合条件によるフィルタリング
# 2023年1月の100ドル以上の販売を製品別に集計
cur.execute("""
-- 複合条件によるフィルタリング
-- 2023年1月の100ドル以上の販売を製品別に集計
SELECT
    product_id,
    SUM(quantity) AS total_quantity,
    SUM(amount) AS total_amount
FROM iceberg.retail.sales_by_date
WHERE sale_date BETWEEN DATE '2023-01-01' AND DATE '2023-01-31'
  AND amount > 100.00
GROUP BY product_id
""")
product_summary = cur.fetchall()
product_columns = [desc[0] for desc in cur.description]
pd.DataFrame(product_summary, columns=product_columns)

### 2. データの更新とタイムトラベルクエリの実行

Iceberg の重要な機能の一つであるタイムトラベルを使って、テーブルの過去のバージョンにアクセスする方法を見ていきましょう。

In [ ]:
# タイムトラベルのデモ用テーブル作成
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.time_travel_demo (
    id INTEGER,
    name VARCHAR,
    value DOUBLE
)
""")

# 最初のデータセットを挿入
cur.execute("""
INSERT INTO iceberg.retail.time_travel_demo VALUES
    (1, 'Item A', 10.5),
    (2, 'Item B', 20.75),
    (3, 'Item C', 15.25)
""")

# 現在のタイムスタンプを記録
cur.execute("SELECT current_timestamp")
first_timestamp = cur.fetchone()[0]
print(f"データセット挿入時刻: {first_timestamp}")

In [ ]:
# テーブルスナップショットの確認
cur.execute("SELECT * FROM iceberg.retail.\"time_travel_demo$snapshots\"")
snapshots_after_first = cur.fetchall()
first_snapshot_id = snapshots_after_first[1][1]  # snapshot_id
print(f"データセット挿入時のスナップショットID: {first_snapshot_id}")

In [ ]:
# 2つ目のデータセットを挿入
cur.execute("""
INSERT INTO iceberg.retail.time_travel_demo VALUES
    (4, 'Item D', 30.0),
    (5, 'Item E', 25.5)
""")

# 現在のテーブルデータを確認
cur.execute("SELECT * FROM iceberg.retail.time_travel_demo ORDER BY id")
current_data = cur.fetchall()
current_columns = [desc[0] for desc in cur.description]
pd.DataFrame(current_data, columns=current_columns)

In [ ]:
# タイムスタンプによるタイムトラベル
cur.execute(f"""
SELECT * FROM iceberg.retail.time_travel_demo FOR TIMESTAMP AS OF TIMESTAMP '{first_timestamp}'
ORDER BY id
""")
first_version_by_time = cur.fetchall()
pd.DataFrame(first_version_by_time, columns=current_columns)

In [ ]:
# スナップショットIDによるタイムトラベル
cur.execute(f"""
SELECT * FROM iceberg.retail.time_travel_demo FOR VERSION AS OF {first_snapshot_id}
ORDER BY id
""")
first_version_by_id = cur.fetchall()
pd.DataFrame(first_version_by_id, columns=current_columns)

In [ ]:
# スナップショット履歴の確認
cur.execute("SELECT * FROM iceberg.retail.\"time_travel_demo$history\"")
history = cur.fetchall()
history_columns = [desc[0] for desc in cur.description]
pd.DataFrame(history, columns=history_columns)

### 3. 高度なWindowクエリを使用した分析

ビジネスデータの分析において、「全体の集計値を知りたいが、同時に個々のデータの詳細も維持したい」というニーズはよくあります。通常の集計関数（SUM、AVG、COUNTなど）では、データをグループ化すると詳細情報が失われてしまいます。この課題を解決するのがウィンドウ関数です。

ウィンドウ関数とは、データの「窓」（特定の行の集合）に対して計算を行い、各行に対して結果を返す特殊な関数です。通常の集計関数と異なり、行をグループに縮約することなく、もとの各行に対して計算結果を付加できます。ウィンドウ関数自体はIceberg固有の機能ではありませんが、Icebergのテーブルへの実践的な分析を行ううえで便利な機能です。

Trinoは豊富なウィンドウ関数をサポートしています。ウィンドウ関数を理解するには、それを構成する3つの要素を知ることが重要です。

- **パーティション（`PARTITION BY`）**：データをどのように分割するか
- **順序（`ORDER BY`）**：各パーティション内でデータをどのように並べるか
- **フレーム（`ROWS/RANGE BETWEEN`）**：現在の行に対して、どの範囲の行を計算に含めるか

それでは、実際の例を通してウィンドウ関数の活用法を見ていきましょう。まず、分析用のサンプルデータを作成します。

In [ ]:
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.detailed_sales (
    sale_id VARCHAR,
    sale_date DATE,
    product_id VARCHAR,
    product_category VARCHAR,
    store_id VARCHAR,
    sales_person_id VARCHAR,
    quantity INTEGER,
    unit_price DECIMAL(10, 2),
    discount DECIMAL(5, 2),
    total_price DECIMAL(10, 2)
)
""")

# サンプルデータの挿入
cur.execute("""
INSERT INTO iceberg.retail.detailed_sales VALUES
    ('S-1001', DATE '2023-01-05', 'P-101', 'Electronics', 'Store-A', 'SP-1', 2, 500.00, 0.00, 1000.00),
    ('S-1002', DATE '2023-01-10', 'P-102', 'Home', 'Store-B', 'SP-2', 1, 300.00, 30.00, 270.00),
    ('S-1003', DATE '2023-01-15', 'P-101', 'Electronics', 'Store-A', 'SP-1', 1, 500.00, 50.00, 450.00),
    ('S-1004', DATE '2023-01-20', 'P-103', 'Clothing', 'Store-C', 'SP-3', 3, 100.00, 0.00, 300.00),
    ('S-1005', DATE '2023-01-25', 'P-104', 'Home', 'Store-B', 'SP-2', 2, 250.00, 25.00, 475.00),
    ('S-1006', DATE '2023-02-05', 'P-101', 'Electronics', 'Store-A', 'SP-1', 1, 520.00, 0.00, 520.00),
    ('S-1007', DATE '2023-02-10', 'P-105', 'Clothing', 'Store-C', 'SP-3', 4, 80.00, 0.00, 320.00),
    ('S-1008', DATE '2023-02-15', 'P-102', 'Home', 'Store-B', 'SP-4', 2, 300.00, 60.00, 540.00),
    ('S-1009', DATE '2023-02-20', 'P-103', 'Clothing', 'Store-A', 'SP-1', 1, 100.00, 10.00, 90.00),
    ('S-1010', DATE '2023-02-25', 'P-104', 'Home', 'Store-B', 'SP-2', 3, 250.00, 0.00, 750.00)
""")

「各カテゴリで最も売れている製品は何か？」を調査するユースケースを考えてみましょう。ウィンドウ関数を使うことで、各製品の売上を集計し、カテゴリ内でのランキングを付けられます。

このクエリのうち`RANK() OVER (PARTITION BY product_category ORDER BY total_sales DESC)`という部分がウィンドウ関数で、以下の処理が行われています。

- `PARTITION BY product_category`：データを製品カテゴリごとに分割する
- `ORDER BY SUM(total_price) DESC`：各カテゴリ内で、総売上高の降順に並べる
- `RANK()`：並べられた順に順位を割り当てる（同点の場合は同じ順位になり、次の順位はスキップされる）

結果として、各製品に対して、そのカテゴリ内での売上ランキングが付与されます。これにより、「`product_category`が`Electronics`の中で最も売れている製品は何か？」といった質問に即座に答えられます。

In [ ]:
cur.execute("""
WITH product_sales AS (
    SELECT
        product_id,
        product_category,
        SUM(total_price) AS total_sales  -- 全期間の合計
    FROM iceberg.retail.detailed_sales
    GROUP BY product_id, product_category
)
SELECT
    product_id,
    product_category,
    total_sales,
    RANK() OVER (
        PARTITION BY product_category
        ORDER BY total_sales DESC
    ) AS category_rank
FROM product_sales
""")

rank_data = cur.fetchall()
rank_columns = [desc[0] for desc in cur.description]
rank_df = pd.DataFrame(rank_data, columns=rank_columns)
print("\n製品カテゴリ別・月次ランキング:")
print(rank_df)

日々の売上のトレンドを把握するために、移動平均を計算してみましょう。移動平均は、短期的な変動を平滑化し、長期的なトレンドを把握するのに役立ちます。

このクエリでは、`AVG(daily_total) OVER (ORDER BY sale_date ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)`の部分でウィンドウ関数が使用されており、以下の処理が行われています。

- `ORDER BY sale_date`：日付順にデータを並べる
- `ROWS BETWEEN 2 PRECEDING AND CURRENT ROW`：現在の行と、その前の2行（合計3行）をウィンドウフレームとして定義する
- `AVG(daily_total)`：このフレーム内の日次売上の平均を計算する

たとえば、1月15日の行を処理する場合、1月5日、1月10日、1月15日の3日間の売上平均が計算されます。データの初期部分（最初の行など）では、前の行が足りない場合、利用可能な行だけで平均を計算します。この移動平均により、日々の短期的な変動の影響を緩和して、売上の全体的な傾向を把握できます。特に季節変動や週末効果がある小売データで有用です。

In [ ]:
cur.execute("""
WITH daily_sales AS (
    SELECT
        sale_date,
        SUM(total_price) AS daily_total
    FROM iceberg.retail.detailed_sales
    GROUP BY sale_date
)
SELECT
    sale_date,
    daily_total,
    AVG(daily_total) OVER (
        ORDER BY sale_date
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS moving_avg_3day
FROM daily_sales
ORDER BY sale_date
""")

data = cur.fetchall()
trend_columns = [desc[0] for desc in cur.description]
trend_df = pd.DataFrame(data, columns=trend_columns)
print("\n移動平均:")
print(trend_df)

製品ごとの累積売上を追跡するために、ウィンドウ関数を使用して各カテゴリの売上を日付ごとに累積してみましょう。これにより、特定の期間における各カテゴリの売上成長を把握できます。

このクエリのウィンドウ関数`SUM(daily_sales) OVER (PARTITION BY product_category ORDER BY sale_date ROWS UNBOUNDED PRECEDING)`は以下のようになっています。

- `PARTITION BY product_category`：カテゴリごとに独立して累積を計算する
- `ORDER BY sale_date`：日付順にデータを並べる
- `ROWS UNBOUNDED PRECEDING`：「パーティションの先頭から現在の行まで」をフレームとして定義する
- `SUM(daily_sales)`：このフレーム内の売上合計を計算する

たとえば、「Electronics」カテゴリの2月5日の行では、1月5日から2月5日までのElectronics製品の総売上が表示されます。これにより、各カテゴリの売上成長を時系列で追跡できます。

製品ごとの累積売上を追跡するために、ウィンドウ関数を使用して各カテゴリの売上を日付ごとに累積してみましょう。これにより、特定の期間における各カテゴリの売上成長を把握できます。

In [ ]:
cur.execute("""
WITH date_sales AS (
    SELECT
        sale_date,
        product_category,
        SUM(total_price) AS daily_sales
    FROM iceberg.retail.detailed_sales
    GROUP BY sale_date, product_category
)
SELECT
    sale_date,
    product_category,
    daily_sales,
    SUM(daily_sales) OVER (
        PARTITION BY product_category
        ORDER BY sale_date
        ROWS UNBOUNDED PRECEDING
    ) AS cumulative_sales
FROM date_sales
ORDER BY product_category, sale_date
""")

data = cur.fetchall()
category_sales_columns = [desc[0] for desc in cur.description]
category_sales_df = pd.DataFrame(data, columns=rank_columns)
print("\n製品ごとの累積売り上げ:")
print(category_sales_df)

最後に、製品ごとの価格変動を分析してみましょう。これは価格戦略の効果を評価するのに役立ちます。

このクエリでは、`LAG(unit_price, 1) OVER (PARTITION BY product_id ORDER BY sale_date)`というウィンドウ関数を使用しています。

- `PARTITION BY product_id`：製品ごとに独立して計算する
- `ORDER BY sale_date`：日付順にデータを並べる
- `LAG(unit_price, 1)`：現在の行より1行前の`unit_price`値を取得する

たとえば、製品P-101の2月5日の行では、前回の販売日（1月15日）の価格が`previous_price`列に表示され、その差額が`price_change`列に計算されます。これにより、時間経過に伴う価格変動を簡単に追跡できます。

最後に、製品ごとの価格変動を分析してみましょう。これは価格戦略の効果を評価するのに役立ちます。

In [ ]:
cur.execute("""
SELECT
    sale_date,
    product_id,
    unit_price,
    LAG(unit_price, 1) OVER (PARTITION BY product_id ORDER BY sale_date) AS previous_price,
    unit_price - LAG(unit_price, 1) OVER (PARTITION BY product_id ORDER BY sale_date) AS price_change
FROM iceberg.retail.detailed_sales
ORDER BY product_id, sale_date
""")

data = cur.fetchall()
price_columns = [desc[0] for desc in cur.description]
price_df = pd.DataFrame(data, columns=price_columns)
print("\n製品ごとの累積売り上げ:")
print(price_df)

### 4. メタデータテーブルの探索

Iceberg はテーブルに関する多くのメタデータを提供しており、これを参照することでテーブルの状態や履歴を把握できます。以下の例では、Iceberg テーブルのメタデータテーブルをクエリして、テーブルの内部構造、履歴、ファイル情報などを表示しています。これらのメタデータは、テーブル管理やトラブルシューティングに役立ちます。メタデータテーブルにアクセスする際は、テーブル名に特殊なサフィックスを付けてクエリします。例えば、`table_name$properties` や `table_name$snapshots` などです。

In [ ]:
# Properties メタデータテーブル
print("Properties メタデータテーブル:")
print("=" * 50)
try:
    cur.execute("SELECT * FROM iceberg.retail.\"sales_by_date$properties\" LIMIT 10")
    data = cur.fetchall()
    if data:
        columns = [desc[0] for desc in cur.description]
        df = pd.DataFrame(data, columns=columns)
        display(df)
    else:
        print("データがありません")
except Exception as e:
    print(f"エラー: {e}")

In [ ]:
# History メタデータテーブル
print("History メタデータテーブル:")
print("=" * 50)
try:
    cur.execute("SELECT * FROM iceberg.retail.\"sales_by_date$history\" LIMIT 10")
    data = cur.fetchall()
    if data:
        columns = [desc[0] for desc in cur.description]
        df = pd.DataFrame(data, columns=columns)
        display(df)
    else:
        print("データがありません")
except Exception as e:
    print(f"エラー: {e}")

In [ ]:
# Metadata Log Entries メタデータテーブル
print("Metadata Log Entries メタデータテーブル:")
print("=" * 50)
try:
    cur.execute("SELECT * FROM iceberg.retail.\"sales_by_date$metadata_log_entries\" LIMIT 10")
    data = cur.fetchall()
    if data:
        columns = [desc[0] for desc in cur.description]
        df = pd.DataFrame(data, columns=columns)
        display(df)
    else:
        print("データがありません")
except Exception as e:
    print(f"エラー: {e}")

In [ ]:
# Snapshots メタデータテーブル
print("Snapshots メタデータテーブル:")
print("=" * 50)
try:
    cur.execute("SELECT * FROM iceberg.retail.\"sales_by_date$snapshots\" LIMIT 10")
    data = cur.fetchall()
    if data:
        columns = [desc[0] for desc in cur.description]
        df = pd.DataFrame(data, columns=columns)
        display(df)
        
        # スナップショットの詳細情報を見やすく表示
        print("\nスナップショットの概要:")
        print(f"総スナップショット数: {len(data)}")
        if len(data) > 0:
            print(f"最新スナップショット ID: {df['committed_at'].max()}")
    else:
        print("データがありません")
except Exception as e:
    print(f"エラー: {e}")

In [ ]:
# Manifests メタデータテーブル
print("Manifests メタデータテーブル:")
print("=" * 50)
try:
    cur.execute("SELECT * FROM iceberg.retail.\"sales_by_date$manifests\" LIMIT 10")
    data = cur.fetchall()
    if data:
        columns = [desc[0] for desc in cur.description]
        df = pd.DataFrame(data, columns=columns)
        display(df)
    else:
        print("データがありません")
except Exception as e:
    print(f"エラー: {e}")

In [ ]:
# All Manifests メタデータテーブル
print("All Manifests メタデータテーブル:")
print("=" * 50)
try:
    cur.execute("SELECT * FROM iceberg.retail.\"sales_by_date$all_manifests\" LIMIT 10")
    data = cur.fetchall()
    if data:
        columns = [desc[0] for desc in cur.description]
        df = pd.DataFrame(data, columns=columns)
        display(df)
    else:
        print("データがありません")
except Exception as e:
    print(f"エラー: {e}")

In [ ]:
# Partitions メタデータテーブル
print("Partitions メタデータテーブル:")
print("=" * 50)
try:
    cur.execute("SELECT * FROM iceberg.retail.\"sales_by_date$partitions\" LIMIT 10")
    data = cur.fetchall()
    if data:
        columns = [desc[0] for desc in cur.description]
        df = pd.DataFrame(data, columns=columns)
        display(df)
        
        # パーティションの統計情報
        print("\nパーティション統計:")
        print(f"総パーティション数: {len(data)}")
    else:
        print("データがありません")
except Exception as e:
    print(f"エラー: {e}")

In [ ]:
# Files メタデータテーブル
print("Files メタデータテーブル:")
print("=" * 50)
try:
    cur.execute("SELECT * FROM iceberg.retail.\"sales_by_date$files\" LIMIT 10")
    data = cur.fetchall()
    if data:
        columns = [desc[0] for desc in cur.description]
        df = pd.DataFrame(data, columns=columns)
        display(df)
        
        # ファイルサイズの統計
        if 'file_size_in_bytes' in columns:
            print("\nファイルサイズ統計:")
            print(f"総ファイル数: {len(data)}")
            print(f"平均ファイルサイズ: {df['file_size_in_bytes'].mean():.2f} bytes")
    else:
        print("データがありません")
except Exception as e:
    print(f"エラー: {e}")

In [ ]:
# Entries メタデータテーブル
print("Entries メタデータテーブル:")
print("=" * 50)
try:
    cur.execute("SELECT * FROM iceberg.retail.\"sales_by_date$entries\" LIMIT 10")
    data = cur.fetchall()
    if data:
        columns = [desc[0] for desc in cur.description]
        df = pd.DataFrame(data, columns=columns)
        display(df)
    else:
        print("データがありません")
except Exception as e:
    print(f"エラー: {e}")

In [ ]:
# All Entries メタデータテーブル
print("All Entries メタデータテーブル:")
print("=" * 50)
try:
    cur.execute("SELECT * FROM iceberg.retail.\"sales_by_date$all_entries\" LIMIT 10")
    data = cur.fetchall()
    if data:
        columns = [desc[0] for desc in cur.description]
        df = pd.DataFrame(data, columns=columns)
        display(df)
    else:
        print("データがありません")
except Exception as e:
    print(f"エラー: {e}")

In [ ]:
# Refs メタデータテーブル
print("Refs メタデータテーブル:")
print("=" * 50)
try:
    cur.execute("SELECT * FROM iceberg.retail.\"sales_by_date$refs\" LIMIT 10")
    data = cur.fetchall()
    if data:
        columns = [desc[0] for desc in cur.description]
        df = pd.DataFrame(data, columns=columns)
        display(df)
        
        # 参照の種類を表示
        if 'type' in columns:
            print("\n参照タイプ:")
            print(df['type'].value_counts())
    else:
        print("データがありません")
except Exception as e:
    print(f"エラー: {e}")

## 書き込みオペレーション

### 1. データの更新と削除

Iceberg テーブルのデータを更新・削除する例を見てみましょう。ここでは、`INSERT INTO` ステートメントを使用して新しいデータを追加し、`UPDATE` と `DELETE` ステートメントを使用して既存のデータを変更します。  

In [ ]:
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.inventory (
    product_id VARCHAR,
    name VARCHAR,
    category VARCHAR,
    stock INTEGER,
    unit_price DECIMAL(10, 2),
    last_updated TIMESTAMP
)
""")

# サンプルデータの挿入
cur.execute("""
INSERT INTO iceberg.retail.inventory VALUES
    ('P001', 'Laptop', 'Electronics', 50, 999.99, TIMESTAMP '2023-01-01 00:00:00'),
    ('P002', 'Smartphone', 'Electronics', 100, 499.99, TIMESTAMP '2023-01-01 00:00:00'),
    ('P003', 'Headphones', 'Electronics', 200, 99.99, TIMESTAMP '2023-01-01 00:00:00'),
    ('P004', 'Coffee Maker', 'Kitchen', 30, 79.99, TIMESTAMP '2023-01-01 00:00:00'),
    ('P005', 'Blender', 'Kitchen', 45, 49.99, TIMESTAMP '2023-01-01 00:00:00')
""")

# 初期データの確認
cur.execute("SELECT * FROM iceberg.retail.inventory ORDER BY product_id")
initial_data = cur.fetchall()
columns = [desc[0] for desc in cur.description]
pd.DataFrame(initial_data, columns=columns)

In [ ]:
# データの更新: 価格の調整と在庫の更新
cur.execute("""
UPDATE iceberg.retail.inventory
SET 
    stock = stock - 10,
    unit_price = unit_price * 0.9,  -- 10%割引
    last_updated = current_timestamp
WHERE category = 'Electronics'
""")

In [ ]:
# 特定の製品の削除
cur.execute("""
DELETE FROM iceberg.retail.inventory
WHERE product_id = 'P005'
""")

In [ ]:
# 更新後のデータを確認
cur.execute("SELECT * FROM iceberg.retail.inventory ORDER BY product_id")
updated_data = cur.fetchall()
pd.DataFrame(updated_data, columns=columns)

In [ ]:
# テーブル履歴の確認
cur.execute("SELECT * FROM iceberg.retail.\"inventory$history\"")
history = cur.fetchall()
history_columns = [desc[0] for desc in cur.description]
history_df = pd.DataFrame(history, columns=history_columns)
print("\nテーブル変更履歴:")
print(history_df)

### 2. MERGEによるUpsert操作

MERGEステートメントは、条件に基づいてデータの更新と挿入を一度に行う強力な機能です。典型的なETLパイプラインでよく使用されます。

In [ ]:
# 顧客データテーブルの作成
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.customers (
    customer_id BIGINT,
    name VARCHAR,
    email VARCHAR,
    status VARCHAR,
    last_update TIMESTAMP
)
""")

# 初期データの挿入
cur.execute("""
INSERT INTO iceberg.retail.customers VALUES
    (101, '田中 太郎', 'tanaka@example.com', 'ACTIVE', TIMESTAMP '2023-01-01 10:00:00'),
    (102, '佐藤 花子', 'sato@example.com', 'ACTIVE', TIMESTAMP '2023-01-01 10:00:00'),
    (103, '鈴木 一郎', 'suzuki@example.com', 'INACTIVE', TIMESTAMP '2023-01-01 10:00:00')
""")

# 更新用テーブルの作成
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.customer_updates (
    customer_id BIGINT,
    name VARCHAR,
    email VARCHAR,
    status VARCHAR
)
""")

# 更新データの挿入（既存顧客の更新と新規顧客）
cur.execute("""
INSERT INTO iceberg.retail.customer_updates VALUES
    (102, '佐藤 花子', 'sato.new@example.com', 'ACTIVE'),  -- メールアドレス更新
    (103, '鈴木 一郎', 'suzuki@example.com', 'ACTIVE'),    -- ステータス更新
    (104, '伊藤 誠', 'ito@example.com', 'ACTIVE')          -- 新規顧客
""")

# 元のデータを確認
cur.execute("SELECT customer_id, name, email, status FROM iceberg.retail.customers ORDER BY customer_id")
original_data = cur.fetchall()
original_columns = [desc[0] for desc in cur.description]
original_df = pd.DataFrame(original_data, columns=original_columns)
print("元の顧客データ:")
print(original_df)

# 更新データを確認
cur.execute("SELECT * FROM iceberg.retail.customer_updates ORDER BY customer_id")
updates = cur.fetchall()
update_columns = [desc[0] for desc in cur.description]
updates_df = pd.DataFrame(updates, columns=update_columns)
print("\n更新用データセット:")
print(updates_df)

# MERGE操作を実行
cur.execute("""
MERGE INTO iceberg.retail.customers c
USING iceberg.retail.customer_updates u
ON c.customer_id = u.customer_id
WHEN MATCHED THEN
    UPDATE SET 
        email = u.email,
        status = u.status,
        last_update = TIMESTAMP '2023-03-30 10:30:45.123'
WHEN NOT MATCHED THEN
    INSERT (customer_id, name, email, status, last_update)
    VALUES (u.customer_id, u.name, u.email, u.status, TIMESTAMP '2023-03-30 10:30:45.123')
""")

# MERGE後のデータを確認
cur.execute("SELECT * FROM iceberg.retail.customers ORDER BY customer_id")
merged_data = cur.fetchall()
merged_columns = [desc[0] for desc in cur.description]
merged_df = pd.DataFrame(merged_data, columns=merged_columns)
print("\nMERGE実行後の顧客データ:")
print(merged_df)